# REGRESION LGISTICA DETECCION DE SPAM

en este ejersici se muestra los fundamentos de la regrecion logistica planteando uno de los primeros problemas que fueron solucionados mediante tecnicas de machinlearning: Deteccion de SPAM

## enunciado del ejersicio 

se propone la construccion de un sistema automatico capaz de predecir si un correo determinado es un correo SPAM o no para ello se utilizara el siguiente Dataset 

[2007 TREC Public Spam corpus](https://terabox.com/s/1QwAgmiBvDVTBspietF6K8w)

la carpeta contiene 75,419 mensajes de correo electronico 

25220 son HAM
50199 son SPAM 

These messages constitute al the messages delivered to aparticula server bitwween these dates:/*312

sun, 8 Appr 2007 13:07:21 - 0400
fri 6 jul 2007 07:04

* Aprendizaje **supervisado**.
* Aprendizaje basado en modelos**
* se corresponde con un **modelo lineal generalizadp.
* realiza predicciones computando una **suama ponderada de las caracteristicas de entrada** sumandole una constante conocida como bias, pero se aplica una funcion logistica al resultado 

## Funciones complementarias 

En este caso práctico con la deteccion de correos electronicos SPAM, el conjunto de datos de que se dispone esta tomado por correos electronicos con sus correcpondientes cabeceras y campos adicionales por lo tanto se requieren de un procesamiento previo a que sean injeridos por el algoritmo de Machine Learning 

In [1]:
# Esta clase facilita el preprocesamiento de correos electronicos que possen codigo HTML
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []
        
    def handle_data(self, d):
        self.fed.append(d)
        
    def get_data(self):
        return ''.join(self.fed)
        
        

In [2]:
# funcion que se encargara de eliminar tags html que se encuentren en el texto del correo 
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
# ejemplo de eliminacion de los tags HTML de un texto 
t = '<tr><td align    ="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>'
strip_tags(t)

'Phrack World News'

Ademas de eliminar los posibles tags HTML que se encuentren en el correo electronico deben
realizarse otras acciones de procesamiento para evitar que los mensajes contengan ruido
necesario, entre ellas se encuentra la eliminacion de los signos de puntuacion, eliminacion 
de posibles campos de correo electronico que no son relevantes o eliminacion de los afijos de una palabra
manteniendo unicaamente la raiz de la misma (Stemming). La clase que se muestra a continuacion realiza estas transformaciones

In [4]:
import email
import string
import nltk 

class Parser:

    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
       
    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Exctact the email content.""" 
        
        subject = self.tokenize(msg['Subject']) if msg['subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                  msg.get_content_type())
        content_type = msg.get_content_type()
        #retornar el content del email 
        return{"subject": subject,
               "body": body,
               "content_type":content_type}
  
    def get_email_body(self, payload, content_type):
        """Extract the body of email."""
        body = []
        if type(payload) is str and content_type =='text/plain':
            return self.tokenize(payload) 
        elif type(payload) is str and content_type =='text/html':
            return self.tokenize(strip_tags(payload))
                                 
        elif type(payload) is list: 
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                        p.get_content_type())
        return body 
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions,
        clean the punctuation symbols and do stemming of the text."""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
                                 #stemming of the tokens
        return [self.stemmer.stem (w) for w in tokens if w not in self.stopwords]
    

## Lectura de un correo en formato RAW

In [5]:
inmail = open("datasets/trec07p/data/inmail.1").read()
print(inmail)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/trec07p/data/inmail.1'

#### Parsing del correo electronico

In [ ]:
import nltk
nltk.download('stopwords')
p = Parser()
p.parse ("datasets/trec07p/data/inmail.1")

#### lectura del indice

esta funcion complementaria se encarga de cargar en memoria la ruta de cada correo electronico y su etiqueta correspondiente {spam,ham}

In [ ]:
index = open("datasets/trec07p/full/index").readlines()
index

In [ ]:
import os 
DATASET_PATH = 'datasets/trec07p'
def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split("../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label":label, "email_path":os.path.join(DATASET_PATH, path)})
    return ret_indexes

In [ ]:
def parse_email(index):
    p= Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index ["label"]

In [ ]:
indexes= parse_index("datasets/trec07p/full/index", 10)
indexes

### Pre procesamiento de los datos del set de Datos

con las funciones precentadas anteriormente se permite la lectura de los correos electronicos de manera programatica y el procesamiento de los mismos para eliminar quellos componenetes que no son de utilidad para la deteccion de los correos spam sin embargo cada uno de los correos sige estando reprecentado por un diccionario de python con una serie de palabras.
 


In [ ]:
# cargar el indice y las etiquetas en memoria 
index = parse_index("datasets/trec07p/full/index",1)

In [ ]:
# leer el primer correo electronico 
import os 

open (index[0]["email_path"]).read()

In [ ]:
# parsear el primer correo 
mail, label = parse_email(index[0])
print("el correo es:", label)
print(mail)

El algoritmo de regreción logística no es capaz de ingerir texto como parte del conjunto de datos de los correos electrinicos parseados en una representacion numérica.

##### Aplicación de CountVectorizer.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Asumiendo que email y mail son objetos válidos
email = {
    "subject": ["subject_text"],
    "body": ["body_text"]
}

prep_email = [" ".join(email["subject"]) + " ".join(email["body"])]

vectorizer = CountVectorizer() 
X = vectorizer.fit(prep_email)

print("e-mail: ", prep_email, "\n")
print("Características de entrada: ", vectorizer.get_feature_names_out())


e-mail:  ['subject_textbody_text'] 

Características de entrada:  ['subject_textbody_text']


In [9]:
X = vectorizer.transform(prep_email)
print("\nValues: \n", X.toarray())


Values: 
 [[1]]


#### Aplicación de OneHotEncoding

In [11]:
from sklearn.preprocessing import OneHotEncoder

# Definir la variable 'mail' con los datos adecuados
mail = {
    'subject': ['subject_text'],
    'body': ['body_text']
}

prep_email = [[w] for w in mail['subject'] + mail['body']]

enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(prep_email)

print("Features:\n", enc.get_feature_names_out())
print("\nValues:\n", X.toarray())


Features:
 ['x0_body_text' 'x0_subject_text']

Values:
 [[0. 1.]
 [1. 0.]]


##### Fuciones auxiloares para el preprocesamiento del conjunto de datos.

In [12]:
def create_prep_dataset(index_path, n_elements):
    x = []
    y = []
    indexes = parse_index(index_path, n_elements)
    for i in range(n_elements):
        print("\rParsing email: {0}".format(i+1), end = '')
        mail, label = parse_email(indexes[i])
        x.append(" ".join(mail['subject']) + " ".join(mail['body']))
        y.append(label)
    return x, y

## 3.- Entenamiento del algoritmo.

In [13]:
# Leer únicamente un subconnunto de 100 correos electrónicos
x_train, y_train = create_prep_dataset("datasets/trec07p/full/index", 100)
x_train

NameError: name 'parse_index' is not defined

#####  Vectorización de los datos.

In [ ]:
Vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)

In [ ]:
print(x_train.toarray())
print("\nFeature:", len(vectorizer.get_feature_names_out()))

In [ ]:
import pandas as pd
pd.DataFrame(x_train.toarray(), columns=[vectorizer.get_feature_names_out()])

In [ ]:
y_train

##### Entrenamiento de regrecion logistica con el conjunto de datos procesados 


In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(x_train, y_train)

#### 4.- Predicción.

#### Lectura del conjunto de correos 

In [ ]:
# Leer 150 correos del datset y quedarse únicamente con los 50 últimos, estos 50 coreos electrónicosno se han utilizado para entrenar el algoritmo.
x,y = create_prep_dataset("datasets/trec07p/full/index", 150)
x_test = x[100:]
y_test = y[100:]


##### Preprocesamiento de los correos con el vectorizador creado anteriormente.

In [ ]:
x_test = vectorizer.transform(x_test)

#### Prediccion del tipo de correo.

In [ ]:
y_pred = clf.predict(x_test)
y_pred

In [ ]:
print("Predicción: \n",y_pred)
print("Etiquetas Reales: \n", y_test)

##### Evaluacion de resultasdos.

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))